In [1]:
from lxml import html

# from bs4 import BeautifulSoup
from bs4 import BeautifulSoup, NavigableString
import time
import datetime
import threading
from apscheduler.schedulers.background import BackgroundScheduler

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.chrome.options import Options

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec

import json
import os
import sys

import helper

In [2]:
course_schedule = []
NO_WINDOW = False

In [3]:
# def sync_get_element_by_xpath(xpath):
#     wait.until(ec.visibility_of_element_located((By.XPATH, xpath)))
#     return driver.find_element_by_xpath(xpath)

In [4]:
#setup option for chrome profile
# chrome_options = Options()
# chrome_options.add_argument("user-data-dir=/Users/huohsien/Library/Application Support/Google/Chrome/Default/")
options = Options()
if NO_WINDOW:
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-gpu')

#start web driver
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
driver.implicitly_wait(20)

wait = WebDriverWait(driver, 30)

[WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [/Users/huohsien/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache


In [5]:
def sign_in():
    username = driver.find_element_by_id('ctl00_cphContents_txtUsername')
    password = driver.find_element_by_id('ctl00_cphContents_txtPassword')
    username.send_keys("DN20092360")
    password.send_keys("jj1216")

    wait.until(ec.visibility_of_element_located((By.CLASS_NAME, "submit-button")))
    sign_in_btn = driver.find_element_by_class_name("submit-button") 
    sign_in_btn.click()

def log_out():
    click_btn = sync_get_element_by_xpath("//a[contains(text(), 'Logout')]")
#     print("click_btn: ", click_btn)
    click_btn.click()
    
def click_book_for_class():
    click_btn = sync_get_element_by_xpath("//a[contains(text(), 'Book for Class')]")
#     print("click_btn: ", click_btn)
    click_btn.click()

def click_book_this_class_now():
    click_btn = driver.find_element_by_xpath("//a[contains(text(), 'BOOK THIS CLASS NOW')]")
    click_btn.click()

In [6]:
#launch url
base_url = "https://www.trueclassbooking.com.tw/member/search-class.aspx"

In [8]:
driver.get(base_url)
sign_in()

In [ ]:
helper.check_exists_by_xpath(driver, "//di[contains(text(), 'Logout')]")

In [9]:
helper.check_exists_by_xpath(driver, "//a[contains(text(), 'Logout')]")

True

In [ ]:
with open(r'/Users/huohsien/workspace/python/yoga_class_smart_booking/class_schedules/14 September - 20 September 2020.json', "r") as read_file:
    course_schedule = json.load(read_file)

In [ ]:
# course_schedule

In [ ]:
book_datetime = datetime.datetime(2020, 9, 10, 22, 0, 0)
courses_to_be_booked = []

c = helper.search_courses_by_date('Tue', course_schedule)
c = helper.search_courses_by_name('Vinyasa Yoga', c)
c = helper.search_courses_by_teacher('Una', c)

courses_to_be_booked.extend(c)

helper.list_courses_to_be_booked(courses_to_be_booked)

In [ ]:
book_datetime = datetime.datetime.now() + datetime.timedelta(seconds=1)
signin_datetime = book_datetime - datetime.timedelta(seconds=3)

In [ ]:
def scheduled_sign_in():
    sign_in()
    click_book_for_class()
    
def reserve_class():
    for course_to_be_booked in courses_to_be_booked:
        book_url = course_to_be_booked['href']
        driver.get(book_url)
        click_book_this_class_now()
        
    time.sleep(30)
    log_out()

In [ ]:
# Start the scheduleruler
scheduler = BackgroundScheduler()
scheduler.add_job(scheduled_sign_in, 'date', run_date= signin_datetime, args=[])
scheduler.add_job(reserve_class, 'date', run_date= book_datetime, args=[])
scheduler.start()